In [ ]:
import cv2
import numpy as np
import os


print("{}".format(os.getcwd()))

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def extract_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.4, 5)
    
    if faces is ():
        #print("No faces found")
        return None
    elif len(faces) > 1:
        #print("Too many faces found")
        return None

    (x,y,w,h) = faces[0]
    cropped = img[y:y+h, x:x+w]
    
    return cropped

capture = cv2.VideoCapture(0)
count = 0

while True:
    ret, frame = capture.read()
    if extract_face(frame) is not None:
        count = count + 1
        face = cv2.resize(extract_face(frame), (200,200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        filename = ".faces/user/{}.jpg".format(count)
        cv2.imwrite(filename, face)
        
        cv2.putText(face, str(count), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.imshow("Face cropper", face)
    
    else:
        print("Face not found")
        pass
    
    if cv2.waitKey(1) == 13 and count == 100:
        break
        
capture.release()
cv2.destroyAllWindows()
print("Collected faces complete")

/Users/aliumujib/Desktop/RoadToAI/OpenCV/simpleML
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face 

# Train Model

In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import join, isfile

data_path = '.faces/user/'
only_files = [f for f in listdir(data_path) if isfile(join(data_path, f))]

training_data, labels = [],[]

for i,files in enumerate(only_files):
    #print("len {} {}".format(i, len(files)))
    image_path = data_path + only_files[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if images is not None:
        training_data.append(np.asarray(images, dtype=np.uint8))
        labels.append(i)
    else:
        print("Image is none {}".format(image_path))
    
#labels = labels[1:]
labels = np.asarray(labels, dtype = np.uint32)
model = cv2.face.LBPHFaceRecognizer_create()
print("Model type:{}".format((labels.tolist())))
model.train(np.asarray(training_data), np.asarray(labels.tolist()))
print("Model successfully trained")

Image is none .faces/user/.DS_Store
Model type:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 2

# Run Face recognition

In [ ]:
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def extract_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.4, 5)
    
    if faces is ():
        print("No faces found")
        return img,[]
    elif len(faces) > 1:
        print("Too many faces found")
        return img,[]

    (x,y,w,h) = faces[0]
    image = cv2.rectangle(img, (x,y),(x+w, y+h),(0,255,255),2)
    roi = img[y:y+h, x:x+w]
    roi = cv2.resize(roi, (200,200))
    return img, roi

capture = cv2.VideoCapture(0)
count = 0

while True:
    ret, frame = capture.read()
    image, face = extract_face(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        results = model.predict(face)

        if results[1] < 500:
            confidence = int(100* (1 - results[1]/300))
            display_string = "{}% confident it is user".format(confidence)
        
        cv2.putText(image, display_string, (100,120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 120, 150), 2)
        
        if confidence > 75:
            cv2.putText(image, "Unlocked", (250,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 120, 150), 2)
            cv2.imshow("Face recognition",image)
        else:
            cv2.putText(image, "Locked", (250,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 120, 150), 2)
            cv2.imshow("Face recognition",image)

    except:
        cv2.putText(image, "No face found", (250,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 120, 150), 2)
        cv2.putText(image, "Locked", (450,700), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 120, 150), 2)
        cv2.imshow("Face recognition",image)
        pass
    
    
    if cv2.waitKey(1) == 13:
        break



Too many faces found
Too many faces found
Too many faces found
No faces found
No faces found
Too many faces found
Too many faces found
Too many faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
Too many faces found
Too many faces found
Too many faces found
Too many faces found
Too many faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No f